<h2><b>Wind Power Generation forecasting</b></h2>

In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.tsa.stattools as st
import sklearn 
from scipy import stats,signal
from sklearn.linear_model import LinearRegression
from datetime import datetime
import datetime as dt
from scipy import stats,signal
from statsmodels.tsa.exponential_smoothing.ets import ETSModel 
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.stats.diagnostic import acorr_ljungbox
from sklearn.metrics import r2_score,mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from statsmodels.tsa.arima_process import ArmaProcess
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.api import VAR # da vedere se tenere questo
from statsmodels.tsa.statespace.varmax import VARMAX
from statsmodels.tsa.stattools import adfuller, kpss
from statsmodels.stats.stattools import durbin_watson
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn')

<h3><b>Data Acquisition</b></h3>

As first step we convert the dataset in a dataframe pandas and let’s take a first look at how it looks.

In [6]:
Wind_df = pd.read_csv("C:/Users/alesi/Desktop/Models/Progetto Models/WindEnergy.csv")
Wind_df

,utc_timestamp,wind_generation_actual,wind_capacity,temperature
0,2017-01-01 00:00:00+00:00,291760.0,37149.130,-2.117
1,2017-01-02 00:00:00+00:00,295097.0,37152.125,-1.154
2,2017-01-03 00:00:00+00:00,666170.0,37155.208,-0.239
3,2017-01-04 00:00:00+00:00,686581.0,37160.125,0.620
4,2017-01-05 00:00:00+00:00,261759.0,37163.542,-3.908
...,...,...,...,...
1089,2019-12-26 00:00:00+00:00,132739.0,50424.125,2.790
1090,2019-12-27 00:00:00+00:00,146590.0,50427.000,1.327
1091,2019-12-28 00:00:00+00:00,159126.0,50427.292,-1.441
1092,2019-12-29 00:00:00+00:00,349839.0,50434.750,-1.314


The dataframe has 4 columns:
- utc_timestamp : year/month/day from 2017/01/01 to 2019/12/30;
- wind_generation_actual : power generated by the turbines in Germany on a specific day;
- wind_capacity : total amount of power generated through a time interval;
- temperature : mean temperature on a specific day.

<h4><b>Check for null and redundant values</b></h4>

In [13]:
Wind_df.info()
# Check null values
Wind_df.isna().sum()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1094 entries, 0 to 1093
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   utc_timestamp           1094 non-null   object 
 1   wind_generation_actual  1094 non-null   float64
 2   wind_capacity           1094 non-null   float64
 3   temperature             1094 non-null   float64
dtypes: float64(3), object(1)
memory usage: 34.3+ KB


utc_timestamp             0
wind_generation_actual    0
wind_capacity             0
temperature               0
dtype: int64

In [14]:
# Check duplicates
Wind_df[Wind_df.duplicated()]

,utc_timestamp,wind_generation_actual,wind_capacity,temperature


After these two preliminary analyses we can say that there are no null and redundant values.